# Code describing the Viterbi algorithm

We start with defining HMM as a list of states, each state defined as a dictionary. We designed it soi that the match states favors a sequence "ACGT". We also include a dictionary giving realtive positions to the previous state of each type

In [16]:
import numpy as np

profile_hmm = [
    {'type': 'S', 'emission': {},                                       'transition': {'M': 0.9, 'I': 0.05, 'D': 0.05}},    # Start State
    {'type': 'I', 'emission': {'A': 0.2, 'C': 0.3, 'G': 0.3, 'T': 0.2}, 'transition': {'M': 0.9, 'I': 0.1}},                # Insert State 1
    {'type': 'D', 'emission': {},                                       'transition': {'M': 0.9, 'D': 0.1}},                # Delete State 1
    {'type': 'M', 'emission': {'A': 0.6, 'C': 0.1, 'G': 0.2, 'T': 0.1}, 'transition': {'M': 0.9, 'I': 0.05, 'D': 0.05}},    # Match State 1
    {'type': 'I', 'emission': {'A': 0.2, 'C': 0.3, 'G': 0.3, 'T': 0.2}, 'transition': {'M': 0.9, 'I': 0.1}},                # Insert State 2
    {'type': 'D', 'emission': {},                                       'transition': {'M': 0.9, 'D': 0.1}},                # Delete State 2
    {'type': 'M', 'emission': {'A': 0.2, 'C': 0.6, 'G': 0.1, 'T': 0.1}, 'transition': {'M': 0.9, 'I': 0.05, 'D': 0.05}},    # Match State 2
    {'type': 'I', 'emission': {'A': 0.2, 'C': 0.3, 'G': 0.3, 'T': 0.2}, 'transition': {'M': 0.9, 'I': 0.1}},                # Insert State 3
    {'type': 'D', 'emission': {},                                       'transition': {'M': 0.9, 'D': 0.1}},                # Delete State 3
    {'type': 'M', 'emission': {'A': 0.1, 'C': 0.2, 'G': 0.5, 'T': 0.2}, 'transition': {'M': 0.9, 'I': 0.05, 'D': 0.05}},    # Match State 3
    {'type': 'I', 'emission': {'A': 0.2, 'C': 0.3, 'G': 0.3, 'T': 0.2}, 'transition': {'M': 0.9, 'D': 0.1}},                # Insert State 4
    {'type': 'D', 'emission': {},                                       'transition': {'E': 1.0}},                          # Delete State 4
    {'type': 'M', 'emission': {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}, 'transition': {'E': 1.0}},                          # Match State 4
    {'type': 'I', 'emission': {'A': 0.2, 'C': 0.3, 'G': 0.3, 'T': 0.2}, 'transition': {'E': 1.0}},                          # Insert State 5
    {'type': 'E', 'emission': {},                                       'transition': {}},                                  # End State 
]

prev_rel_states = {  # Relative position to previous state of each type
    'S': {},
    'M': {'S': -3, 'M':-3, 'I':-2, 'D':-4},
    'I': {'S': -1, 'M':-1, 'I':0         },
    'D': {'S': -2, 'M':-2,         'D':-3},
    'E': {         'M':-2, 'I':-1, 'D':-3},
}


Then we define the dynamic programming algorithm to compute the Viterbi matrix, and backtracking the optimal path (the Viterbi path) through the model.

In [17]:

def viterbi(profile_hmm, sequence):
    num_states = len(profile_hmm)
    num_bases = len(sequence)

    # Initialize the Viterbi and path matrices
    viterbi_matrix = np.zeros((num_states, num_bases+2))
    viterbi_path = np.zeros((num_states, num_bases+2), dtype=int)

    # Initialize the first column of the Viterbi matrix
    viterbi_matrix[0, 0] = 1.0

    # Fill the Viterbi matrix
    for base_idx in range(1, num_bases+2):
        for state in range(num_states):
            transition_probs = {}
            current_type = profile_hmm[state]['type']  # Is this a 'M', I', or 'D' state?
            isCurrentSilent = not profile_hmm[state]['emission']
            # Get the previous states that can transition to the current state
            prev_abs_states = { t : state + rel for t, rel in prev_rel_states[current_type].items() if (state + rel >= 0) and (t == profile_hmm[state+rel]['type']) and (current_type in profile_hmm[state+rel]['transition'])}
            # Get the previous base index, it is different for silent states (S, E and D)
            prev_abs_base = base_idx if (isCurrentSilent) else base_idx -1  
            for prev_type, prev_abs_state in prev_abs_states.items():
                transition_prob = profile_hmm[prev_abs_state]['transition'][current_type]
                prev_score = viterbi_matrix[prev_abs_state, prev_abs_base]
                transition_probs[prev_abs_state] = transition_prob * prev_score
            if transition_probs:  # Check if the list is not empty
                max_prev_state = max(transition_probs, key=transition_probs.get)
                max_transition_prob = transition_probs[max_prev_state]
                # print(max_prev_state, max_transition_prob)
                if profile_hmm[state]['emission'] and base_idx <= num_bases:
                    emission_prob = profile_hmm[state]['emission'].get(sequence[base_idx-1], 0)
                else:
                    emission_prob = 1.0
                viterbi_matrix[state, base_idx] = max_transition_prob * emission_prob
                viterbi_path[state, base_idx] = max_prev_state
    # Trace back to find the most probable path
    state = num_states - 1
    base_idx = num_bases + 1
    letter='-'
    best_path = []
    while base_idx>=1 and state>0:
        best_path.append((state, profile_hmm[state]['type'], letter ))
        state = viterbi_path[state, base_idx]
        isSilent = not profile_hmm[state]['emission']
        if isSilent:
            letter = '-'
        else:
            base_idx -= 1
            letter = sequence[base_idx-1]
    best_path.append((state, profile_hmm[state]['type'], letter ))
    best_path.reverse()
    return best_path




Lets first try with a "ACGT" sequence. 

In [18]:
decoded_path = viterbi(profile_hmm, 'ACGT')
print("Decoded path:")
for state, type, letter in decoded_path:
    print(f"State {state} of type {type} emitted {letter}")


[[1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00]
 [0.00000e+00 1.00000e-02 3.00000e-04 9.00000e-06 1.80000e-07 1.80000e-08]
 [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00]
 [0.00000e+00 5.40000e-01 9.00000e-04 5.40000e-05 8.10000e-07 1.62000e-07]
 [0.00000e+00 0.00000e+00 8.10000e-03 2.43000e-04 4.86000e-06 4.86000e-07]
 [0.00000e+00 2.70000e-02 4.50000e-05 2.70000e-06 4.05000e-08 8.10000e-09]
 [0.00000e+00 0.00000e+00 2.91600e-01 7.29000e-04 2.18700e-05 4.37400e-06]
 [0.00000e+00 0.00000e+00 0.00000e+00 4.37400e-03 8.74800e-05 8.74800e-06]
 [0.00000e+00 2.70000e-03 1.45800e-02 3.64500e-05 1.09350e-06 2.18700e-07]
 [0.00000e+00 0.00000e+00 4.86000e-03 1.31220e-01 7.87320e-04 7.87320e-05]
 [0.00000e+00 0.00000e+00 0.00000e+00 7.29000e-05 1.31220e-03 3.93660e-05]
 [0.00000e+00 2.70000e-04 1.45800e-03 6.56100e-03 3.93660e-05 3.93660e-06]
 [0.00000e+00 0.00000e+00 4.86000e-04 2.62440e-03 4.72392e-02 1.18098e-03]
 [0.00000e+00 0.00000e+00

We could try a sequence that i slonger than the model.

In [19]:
decoded_path = viterbi(profile_hmm, 'ACAAGT')
print("Decoded path:")
for state, type, letter in decoded_path:
    print(f"State {state} of type {type} emitted {letter}")


[[1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 1.000000e-02 3.000000e-04 6.000000e-06 1.200000e-07
  3.600000e-09 7.200000e-11 7.200000e-12]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00]
 [0.000000e+00 5.400000e-01 9.000000e-04 1.620000e-04 3.240000e-06
  2.160000e-08 3.240000e-10 6.480000e-11]
 [0.000000e+00 0.000000e+00 8.100000e-03 1.620000e-04 3.240000e-06
  9.720000e-08 1.944000e-09 1.944000e-10]
 [0.000000e+00 2.700000e-02 4.500000e-05 8.100000e-06 1.620000e-07
  1.080000e-09 1.620000e-11 3.240000e-12]
 [0.000000e+00 0.000000e+00 2.916000e-01 1.458000e-03 2.916000e-05
  2.916000e-07 8.748000e-09 1.749600e-09]
 [0.000000e+00 0.000000e+00 0.000000e+00 2.916000e-03 5.832000e-05
  1.749600e-06 3.499200e-08 3.499200e-09]
 [0.000000e+00 2.700000e-03 1.458000e-02 7.290000e-05 1.458000e-06
  1.458000e-08 4.374000e-10 8.748000e-11]
 [0.000000e+00 0.00

And a shorter sequence.

In [20]:
decoded_path = viterbi(profile_hmm, 'AGT')
print("Decoded path:")
for state, type, letter in decoded_path:
    print(f"State {state} of type {type} emitted {letter}")

[[1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 1.0000e-02 3.0000e-04 6.0000e-06 6.0000e-07]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 5.4000e-01 1.8000e-03 2.7000e-05 5.4000e-06]
 [0.0000e+00 0.0000e+00 8.1000e-03 1.6200e-04 1.6200e-05]
 [0.0000e+00 2.7000e-02 9.0000e-05 1.3500e-06 2.7000e-07]
 [0.0000e+00 0.0000e+00 4.8600e-02 7.2900e-04 1.4580e-04]
 [0.0000e+00 0.0000e+00 0.0000e+00 4.8600e-04 4.8600e-05]
 [0.0000e+00 2.7000e-03 2.4300e-03 3.6450e-05 7.2900e-06]
 [0.0000e+00 0.0000e+00 1.2150e-02 8.7480e-03 6.5610e-04]
 [0.0000e+00 0.0000e+00 0.0000e+00 1.2150e-04 4.3740e-04]
 [0.0000e+00 2.7000e-04 6.0750e-04 4.3740e-04 3.2805e-05]
 [0.0000e+00 0.0000e+00 4.8600e-04 4.3740e-03 7.8732e-03]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 2.7000e-04 6.0750e-04 4.3740e-03 7.8732e-03]]
Decoded path:
State 0 of type S emitted -
State 3 of type M emitted A
State 5 of type D emitted -
State 9 of type M emitted G
Sta